In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1 区域中心点

### 读取城市网格归属区域文件，分组求区域中心

读取的原始数据文件在同一目录下的train_data内的grid_attr.csv 

一个区域原本是由很多点组成一个举行，可以直接简化成一个形心centroid，也就是一个区域对应一个中心点

```python
def zone_cent(city):
    # 读取初始文件
    zone0=pd.read_csv('train_data/city_%s/grid_attr.csv'%city,header=None,names=['zonex','zoney','zoneid'])
    # 按照区域id分组，因为一个区域的所有点构成简单的矩形，所以求矩形的形心点作为区域中心
    zone1=zone0.groupby('zoneid').apply(lambda g:[(g.zonex.max()+g.zonex.min())/2,(g.zoney.max()+g.zoney.min())/2])
    zone1=pd.DataFrame(zip(*zone1)).T.reset_index()
    zone1.columns=['zoneid','zonex','zoney']
    zone1['city']=city
    return zone1

```

### 遍历读取5个城市的grid_attr.csv，将区域代表点文件存入mydata文件夹内，并打印每个城市的区域个数

```python
for city in list('ABCDE'):
    zone=zone_cent(city)
    print('城市%s有%d个区域'%(city,len(zone)))
    zone.to_csv('mydata/zone_cent_%s.csv'%city,index=False)
```

### 例如选择A城市，转为区域代表点，查看前5行

```python
zone_cent('A').head()
```

In [72]:
# zone_cent('A').head()

,zoneid,zonex,zoney,city
0,0,146.765819,30.242467,A
1,1,146.981358,30.071010,A
2,2,146.999338,30.273759,A
3,3,147.053348,30.242174,A
4,4,147.071238,30.289240,A


***

# 2 区域邻接0-1矩阵 neighbor matrix

邻接矩阵都是基于区域id `zoneid`构造的

采用了两种计算方式：


|方法|特点|保存文件|示例(A城市)|
|---|---|---|---|
|2.1 voronoi|基于邻接（共享边）与否<p><p>每个点不定数量的邻接点<p><p>由于voronoi可能形状怪异，**会忽略貌似对角点的相邻区域**|neigh_matrix_voronoi_city.csv|<img src="mydata/example_neigh_matrix_voronoi_A.png" width=800/>|
|2.2 kNN|基于距离（欧式距离）远近<p><p>每个点固定k个邻接点|neigh_matrix_knearast_city.csv|<img src="mydata/example_neigh_matrix_3nearest_A.png" width=800/>|

## 2.0 自定义函数

### 调用scipy的[coo_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html)，输出稀疏矩阵

In [104]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html
from scipy.sparse import coo_matrix

In [105]:
def get_points_array(city):
    '''
    获取城市的区域中心点对array
    city: 城市代号，用'A','B'等表示
    return: 该城市city的区域中心点array，顺序要和zoneid对应
    '''
    # 读取区域中心点文件
    zone_cent=pd.read_csv('mydata/zone_cent_%s.csv'%city)
    # 提取区域的x和y坐标，为numpy格式
    points= zone_cent.set_index('zoneid')[['zonex','zoney']].values
    return points

In [106]:
def neigh_dict_matrix_df(neigh_dict,zone_num):
    '''
    将邻接字典转为0-1稀疏矩阵
    neigh_dict: 字典dict结构，键为区域id，值为该区域邻接（共享边）的区域id
    zone_num: 该城市的区域个数
    return: 该城市的0-1邻接矩阵，以pandas的DataFrame储存
    '''
    neigh_list=[]
    for k,v in neigh_dict.items():
        for i in v:
            neigh_list.append([k,i])
    # 转为np.array格式
    neigh_array=np.array(neigh_list)
    # 转为稀疏矩阵
    neigh_matrix=coo_matrix((np.ones(len(neigh_list)), # 表示邻接的数值1
                             (neigh_array[:,0],neigh_array[:,1])), # 区域1的id，区域2的id
                            shape=(zone_num,zone_num)) # 矩阵大小：区域数*区域数
    # 转为pandas的DataFrame格式，便于储存
    neigh_matrix_df=pd.DataFrame(neigh_matrix.todense()).astype(int)
    return neigh_matrix_df

## 2.1 voronoi 邻接0-1矩阵

### 调用空间处理库pysal的[Voronoi](https://pysal.org/libpysal/generated/libpysal.weights.Voronoi.html)函数

In [107]:
# https://pysal.org/libpysal/generated/libpysal.weights.Voronoi.html
from libpysal.weights import Voronoi

### 转换过程

In [108]:
def neigh_matrix_voronoi(city):
    '''
    city: 城市代号，用'A','B'等表示
    return: 该城市的0-1邻接矩阵
    '''
    ########## (1) 提取中心点（重复代码） ##########
    points= get_points_array(city)
    zone_num=len(points)
    
    ########## (2) 调用pysal函数提取邻接关系字典 ##########
    # 调用pysal的voronoi函数，得到voronoi对象
    w = Voronoi(points)
    # 得到每个Voronoi区域块的共享边的邻接neighbors关系
    # neigh_dict以字典dict储存: 键为区域id，值为该区域邻接（共享边）的区域id
    neigh_dict=w.neighbors
    
    ########## (3) 将邻接关系字典转为0-1矩阵（重复代码） ###########
    neigh_matrix_df=neigh_dict_matrix_df(neigh_dict,zone_num)
    return neigh_matrix_df

### 将5个城市批量转换保存进csv：没有标题和索引

In [109]:
for city in list('ABCDE'):
    neigh_matrix_df=neigh_matrix_voronoi(city)
    print('%s: %s'%(city,neigh_matrix_df.shape))
    neigh_matrix_df.to_csv('mydata/neigh_matrix_voronoi_%s.csv'%city,
                           index=False,header=False)

A: (118, 118)
B: (30, 30)
C: (135, 135)
D: (75, 75)
E: (34, 34)


### 示例：将A城市转为基于voronoi的邻接矩阵

In [110]:
neigh_matrix_voronoi('A')

,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
114,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,1,0,1,1
116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1


***

## 2.2 kNN邻接0-1矩阵

### 调用pysal的k邻近函数[KNN](https://pysal.org/libpysal/generated/libpysal.weights.KNN.html)

In [111]:
# https://pysal.org/libpysal/generated/libpysal.weights.KNN.html
from libpysal.weights import KNN

In [112]:
def neigh_matrix_knn(city,k):
    '''
    city: 城市代号，用'A','B'等表示
    return: 该城市的0-1邻接矩阵
    '''
    ########## (1) 提取中心点（重复代码） ##########
    points= get_points_array(city)
    zone_num=len(points)
    
    ########## (2) 调用pysal函数提取邻接关系字典 ##########
    # 调用pysal的KNN函数，得到k个最临近的区域id
    # Creates nearest neighbor weights matrix based on k nearest neighbors.
    w=KNN.from_array(points, k) 
    # neigh_dict以字典dict储存: 键为区域id，值为该区域邻接（共享边）的区域id
    neigh_dict=w.neighbors
    
    ########## (3) 将邻接关系字典转为0-1矩阵（重复代码） ###########
    neigh_matrix_df=neigh_dict_matrix_df(neigh_dict,zone_num)
    return neigh_matrix_df

### 将5个城市批量转换保存进csv：没有标题和索引

In [113]:
k=3
for city in list('ABCDE'):
    neigh_matrix_df=neigh_matrix_knn(city,k)
    print('%s: %s'%(city,neigh_matrix_df.shape))
    neigh_matrix_df.to_csv('mydata/neigh_matrix_%dnearest_%s.csv'%(k,city),
                           index=False,header=False)

A: (118, 118)
B: (30, 30)
C: (135, 135)
D: (75, 75)
E: (34, 34)


D:\python\lib\site-packages\libpysal\weights\weights.py:167: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
D:\python\lib\site-packages\libpysal\weights\weights.py:167: UserWarning: The weights matrix is not fully connected: 
 There are 6 disconnected components.
  warnings.warn(message)
D:\python\lib\site-packages\libpysal\weights\weights.py:167: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)
D:\python\lib\site-packages\libpysal\weights\weights.py:167: UserWarning: The weights matrix is not fully connected: 
 There are 3 disconnected components.
  warnings.warn(message)


### 示例：将A城市转为基于3-NN的邻接矩阵

In [114]:
neigh_matrix_knn('A',3)

,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
114,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1


***

## 2.3 最短距离矩阵

### 调用pysal的[distance_matrix](https://pysal.org/libpysal/generated/libpysal.cg.distance_matrix.html)函数

In [115]:
from libpysal.cg import distance_matrix
# https://pysal.org/libpysal/generated/libpysal.cg.distance_matrix.html

### 示例：输出A城市118个区域中心点的最短距离矩阵array

In [116]:
points=get_points_array('A')
d = distance_matrix(points)
np.array(d)

array([[0.        , 0.27541746, 0.23560633, ..., 0.64983269, 0.66639286,
        0.66767734],
       [0.27541746, 0.        , 0.20354514, ..., 0.49048513, 0.49940752,
        0.50642422],
       [0.23560633, 0.20354514, 0.        , ..., 0.41448667, 0.4315087 ,
        0.4323576 ],
       ...,
       [0.64983269, 0.49048513, 0.41448667, ..., 0.        , 0.02355187,
        0.01790064],
       [0.66639286, 0.49940752, 0.4315087 , ..., 0.02355187, 0.        ,
        0.0155955 ],
       [0.66767734, 0.50642422, 0.4323576 , ..., 0.01790064, 0.0155955 ,
        0.        ]])

# 3 合并5个城市的区域信息

## 3.1 区域重新编码zoneid2

In [270]:
neighs_3nn={city:np.array for city in list('ABCDE')}
neighs_vor={city:np.array for city in list('ABCDE')}
lens={city:0 for city in list('ABCDE')}

In [271]:
df=pd.DataFrame()
for city in list('ABCDE'):
    tmp=pd.read_csv('mydata/zone_cent_%s.csv'%city)
    lens[city]=len(tmp)
    df=df.append(tmp)
    neighs_3nn[city]=np.loadtxt("mydata/neigh_matrix_3nearest_%s.csv"%city,delimiter=",")
    neighs_vor[city]=np.loadtxt("mydata/neigh_matrix_voronoi_%s.csv"%city,delimiter=",")

In [272]:
df['zoneid2']=range(len(df))
df.to_csv('mydata/zone_cent_all.csv',index=False)

In [273]:
lens

{'A': 118, 'B': 30, 'C': 135, 'D': 75, 'E': 34}

In [274]:
cs=[]
s=0
for k,v in lens.items():
    s+=v
    cs.append(s)

In [275]:
cs

[118, 148, 283, 358, 392]

In [276]:
def diag_combine_neigh_matrixs(neigh):
    mat=np.zeros([len(df),len(df)])
    mat[:cs[0],:cs[0]]=neigh['A']
    mat[cs[0]:cs[1],cs[0]:cs[1]]=neigh['B']
    mat[cs[1]:cs[2],cs[1]:cs[2]]=neigh['C']
    mat[cs[2]:cs[3],cs[2]:cs[3]]=neigh['D']
    mat[cs[3]:cs[4],cs[3]:cs[4]]=neigh['E']
    return mat

In [277]:
pd.DataFrame(diag_combine_neigh_matrixs(neighs_vor)).to_csv('mydata/neigh_matrix_voronoi_all.csv',index=False,header=False)
pd.DataFrame(diag_combine_neigh_matrixs(neighs_3nn)).to_csv('mydata/neigh_matrix_3nearest_all.csv',index=False,header=False)